install Transformers , perfomr sentiment scoring using BERT , scrap reviews fromm yelp ( BS) and score

BERT from hugging face install


install dependencies
instantiate model
encode and calculate sentiment
collct review from yelp ( scrap)
load reviews to dataframe and score

# Install and Import Dependencies


In [ ]:
#Pytorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers beautifulsoup4 requests pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification # tokenizer for string to numbers
import torch #argmax for highest sequence
import requests # html tags 
from bs4 import BeautifulSoup # extract data from html tags
import re

# Instantiate Model


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

#Encode and Calculate Sentiment


In [ ]:
tokens = tokenizer.encode("i hate this, absolutely the worst" , return_tensors = 'pt') # pt is for pytorch, return_tensors is key ere

In [ ]:
#changed the sentense as below to give positive review and the 
#score below is 5 ( good positive sentiment) while for above line it came as  1 ( which is for bad sentiment)
tokens = tokenizer.encode("it was simply amazing and great" , return_tensors = 'pt')

In [ ]:
tokens

tensor([[  101,   151, 39487, 10372,   117, 35925, 10563, 10103, 43060,   102]])

In [ ]:
#jsut to show decoding of what we encoded above :
tokenizer.decode(tokens[0])

'[CLS] i hate this, absolutely the worst [SEP]'

In [ ]:
result = model(tokens)

In [ ]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.0607,  1.6029, -1.0123, -3.0154, -1.8469]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
result.logits # this is result of passing the tokens via the model - one hot encoded list of scores. 
#the position with highest score represent sthe sentiment rating
#higher the number , better the sentiment , lower the number , bad sentimient, 3 in middle where sentiment not recognized.

tensor([[ 5.0607,  1.6029, -1.0123, -3.0154, -1.8469]],
       grad_fn=<AddmmBackward0>)

 ### This is result of passing the tokens via the model - one hot encoded list of scores. The position with highest score represent sthe sentiment rating higher the number , better the sentiment , lower the number , bad sentimient, 3 in middle where sentiment not recognized.

In [ ]:
int(torch.argmax(result.logits))+1

1

#Collect reviews from yelp using Request and parsing via Beautifulsoup 

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup  = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex}) # p for paragraph and class that is in regex ( comment)
reviews = [result.text for result in results]

In [ ]:
reviews

["Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent.",
 "Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent.",
 'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly',
 'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well',
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the

#load reviews into DataFrame and score 



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review']) # converting as numpy array as that is easy in DF

In [ ]:
df['review'].iloc[0]

"Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent."

In [ ]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors = 'pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[0])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x:sentiment_score(x[:512]))


In [ ]:
df

,review,sentiment
0,Great coffee and vibe. That's all you need. C...,5
1,Great coffee and vibe. That's all you need. C...,4
2,Great food amazing coffee and tea. Short walk ...,5
3,Great staff and food. Must try is the pan fri...,5
4,Ricotta hot cakes! These were so yummy. I ate ...,5
5,I came to Social brew cafe for brunch while ex...,5
6,We came for brunch twice in our week-long visi...,4
7,It was ok. The coffee wasn't the best but it w...,3
8,I went here a little while ago- a beautiful mo...,2
9,This place is a gem. The ambiance is to die fo...,3


In [ ]:
df['review'].iloc[3]

'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well'